In [1]:
import pandas as pd
import numpy as np

In [8]:
raw_df = pd.read_excel('../超市.xls')
raw_df

,订单 Id,利润率,记录数,制造商,产品名称,利润,发货日期,国家,地区,城市,子类别,客户名称,折扣,数量,省/自治区,类别,细分,订单日期,邮寄方式,销售额
0,US-2015-1357144,-0.47,1,Fiskars,"Fiskars 剪刀, 蓝色",-61,2016-04-29,中国,华东,杭州,用品,曾惠,0.4,2,浙江,办公用品,公司,2016-04-27,二级,130
1,CN-2015-1973789,0.34,1,GlobeWeis,"GlobeWeis 搭扣信封, 红色",43,2016-06-19,中国,西南,内江,信封,许安,0.0,2,四川,办公用品,消费者,2016-06-15,标准级,125
2,CN-2015-1973789,0.13,1,Cardinal,"Cardinal 孔加固材料, 回收",4,2016-06-19,中国,西南,内江,装订机,许安,0.4,2,四川,办公用品,消费者,2016-06-15,标准级,32
3,US-2015-3017568,-0.08,1,Kleencut,"Kleencut 开信刀, 工业",-27,2016-12-13,中国,华东,镇江,用品,宋良,0.4,4,江苏,办公用品,公司,2016-12-09,标准级,321
4,CN-2014-2975416,0.40,1,KitchenAid,"KitchenAid 搅拌机, 黑色",550,2015-06-02,中国,中南,汕头,器具,万兰,0.0,3,广东,办公用品,消费者,2015-05-31,二级,1376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9930,CN-2015-4318875,-0.33,1,SanDisk,"SanDisk 记忆卡, 实惠",-315,2016-06-02,中国,华东,义乌,配件,巩光,0.4,3,浙江,技术,消费者,2016-05-28,标准级,944
9931,CN-2015-4318875,0.08,1,Eldon,"Eldon 盘, 蓝色",36,2016-06-02,中国,华东,义乌,收纳具,巩光,0.0,2,浙江,办公用品,消费者,2016-05-28,标准级,448
9932,CN-2015-4318875,-0.37,1,Safco,"SAFCO 折叠椅, 红色",-88,2016-06-02,中国,华东,义乌,椅子,巩光,0.4,1,浙江,家具,消费者,2016-05-28,标准级,240
9933,CN-2015-4318875,-0.33,1,罗技,"罗技 路由器, 耐用",-1617,2016-06-02,中国,华东,义乌,配件,巩光,0.4,7,浙江,技术,消费者,2016-05-28,标准级,4852


In [3]:
def get_name(full_name):
    if full_name.find("个")!= -1 and full_name.find("每")!= -1:
        if full_name.find("Green Bar") != -1:
            index = full_name.find(" ")
            index = full_name.find(" ",index+1,-1)
            key = full_name[index+1:]
        else:
            index = full_name.find(" ")
            key = full_name[index+1:]
    else:
        index = full_name.rfind(" ")
        index = full_name.rfind(' ', 0, index)
        key = full_name[index+1:]
    return index, key

In [4]:
# 提取每个商品对应的品牌种类
def get_category_dict(raw_df):
    category_dict = {}
    
    for i in range(len(raw_df['产品名称'])):
        full_name = raw_df['产品名称'][i]
        index, key = get_name(full_name)
            
        if key not in category_dict.keys():
            category_dict[key] = set()
        
        name = raw_df['产品名称'][i][:index]
        category_dict[key].add(full_name)
        
    return category_dict

In [10]:
def get_product_time_feature(raw_df):
    product_time_dict = {}
    for i in range(len(raw_df['订单 Id'])):
        key = raw_df['产品名称'][i]
        if key not in product_time_dict.keys():
            product_time_dict[key] = {'name':key, 'order_time':[], 'deliver_way':[], 
                                      'discount':[], 'days':[], 'price':[], 'profit':[]}
            
        order_time = raw_df['订单日期'][i]
        deliver_way = raw_df['邮寄方式'][i]
        discount = raw_df['折扣'][i]
        days = (raw_df['发货日期'][i] - raw_df['订单日期'][i]).days
        price = raw_df['销售额'][i] / raw_df['数量'][i]
        profit = raw_df['利润率'][i]
        product_time_dict[key]['order_time'].append(order_time)
        product_time_dict[key]['deliver_way'].append(deliver_way)
        product_time_dict[key]['discount'].append(discount)
        product_time_dict[key]['days'].append(days)
        product_time_dict[key]['price'].append(price)
        product_time_dict[key]['profit'].append(profit)
    
    return product_time_dict

In [11]:
pd.DataFrame(data=get_product_time_feature(raw_df))

,"Fiskars 剪刀, 蓝色","GlobeWeis 搭扣信封, 红色","Cardinal 孔加固材料, 回收","Kleencut 开信刀, 工业","KitchenAid 搅拌机, 黑色","柯尼卡 打印机, 红色","Ibico 订书机, 实惠","SAFCO 扶手椅, 可调","Green Bar 计划信息表, 多色","Stockwell 橡皮筋, 整包",...,"惠普 传真复印机, 彩色","松下 收据打印机, 红色","Bevis 培训桌, 白色","Chromcraft 木桌, 长方形","Lesro 圆桌, 黑色","Barricks 会议桌, 组装","SanDisk 令, 回收","Chromcraft 圆桌, 长方形","Kleencut 大剪刀, 工业","StarTech 计算器, 白色"
name,"Fiskars 剪刀, 蓝色","GlobeWeis 搭扣信封, 红色","Cardinal 孔加固材料, 回收","Kleencut 开信刀, 工业","KitchenAid 搅拌机, 黑色","柯尼卡 打印机, 红色","Ibico 订书机, 实惠","SAFCO 扶手椅, 可调","Green Bar 计划信息表, 多色","Stockwell 橡皮筋, 整包",...,"惠普 传真复印机, 彩色","松下 收据打印机, 红色","Bevis 培训桌, 白色","Chromcraft 木桌, 长方形","Lesro 圆桌, 黑色","Barricks 会议桌, 组装","SanDisk 令, 回收","Chromcraft 圆桌, 长方形","Kleencut 大剪刀, 工业","StarTech 计算器, 白色"
order_time,"[2016-04-27 00:00:00, 2015-06-07 00:00:00, 201...","[2016-06-15 00:00:00, 2016-04-03 00:00:00, 201...","[2016-06-15 00:00:00, 2015-11-08 00:00:00, 201...","[2016-12-09 00:00:00, 2016-02-27 00:00:00, 201...","[2015-05-31 00:00:00, 2015-12-24 00:00:00, 201...","[2014-10-27 00:00:00, 2015-11-01 00:00:00, 201...","[2014-10-27 00:00:00, 2015-09-26 00:00:00, 201...","[2014-10-27 00:00:00, 2015-02-25 00:00:00, 201...","[2014-10-27 00:00:00, 2015-10-12 00:00:00]","[2014-10-27 00:00:00, 2015-12-13 00:00:00, 201...",...,"[2013-12-22 00:00:00, 2014-07-14 00:00:00]",[2016-09-24 00:00:00],[2015-04-30 00:00:00],[2014-03-12 00:00:00],[2013-10-10 00:00:00],[2016-09-09 00:00:00],[2013-06-25 00:00:00],[2016-04-01 00:00:00],[2013-05-17 00:00:00],[2013-05-16 00:00:00]
deliver_way,"[二级, 标准级, 二级, 标准级, 二级, 标准级, 标准级, 二级, 二级, 一级, 标准级]","[标准级, 二级, 二级, 标准级]","[标准级, 一级, 二级, 标准级, 标准级, 标准级, 标准级, 标准级, 二级, 标准级...","[标准级, 二级, 当日, 一级]","[二级, 二级, 二级, 标准级, 一级, 一级, 一级]","[标准级, 标准级, 标准级, 标准级]","[标准级, 标准级, 一级, 标准级, 标准级, 一级, 当日, 标准级, 标准级]","[标准级, 一级, 标准级, 一级, 二级, 标准级, 标准级, 一级]","[标准级, 标准级]","[标准级, 标准级, 标准级, 标准级, 二级]",...,"[一级, 二级]",[标准级],[二级],[标准级],[标准级],[二级],[标准级],[二级],[标准级],[标准级]
discount,"[0.4, 0.4, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 0.4, ...","[0.0, 0.0, 0.0, 0.0]","[0.4, 0.4, 0.0, 0.0, 0.0, 0.0, 0.4, 0.4, 0.0, ...","[0.4, 0.0, 0.4, 0.4]","[0.0, 0.0, 0.0, 0.4, 0.0, 0.4, 0.0]","[0.0, 0.4, 0.0, 0.0]","[0.0, 0.4, 0.0, 0.0, 0.4, 0.0, 0.4, 0.4, 0.4]","[0.0, 0.0, 0.4, 0.4, 0.0, 0.0, 0.1, 0.0]","[0.0, 0.0]","[0.0, 0.0, 0.0, 0.4, 0.0]",...,"[0.0, 0.0]",[0.0],[0.25],[0.25],[0.25],[0.6],[0.0],[0.25],[0.0],[0.4]
days,"[2, 7, 2, 4, 2, 5, 4, 5, 2, 3, 5]","[4, 4, 2, 7]","[4, 3, 4, 6, 4, 7, 4, 4, 2, 4, 2, 4]","[4, 2, 1, 2]","[2, 2, 2, 6, 2, 2, 2]","[4, 4, 6, 4]","[4, 6, 1, 4, 4, 2, 0, 6, 5]","[4, 1, 7, 1, 5, 6, 6, 3]","[4, 4]","[4, 6, 4, 4, 5]",...,"[3, 2]",[4],[5],[4],[6],[5],[5],[4],[4],[6]
price,"[65.0, 64.83333333333333, 108.0, 64.8, 108.0, ...","[62.5, 62.666666666666664, 62.75, 62.5]","[16.0, 16.0, 26.6, 26.666666666666668, 26.5, 2...","[80.25, 133.75, 80.5, 80.0]","[458.6666666666667, 458.6666666666667, 458.5, ...","[1236.6666666666667, 742.0, 1236.6666666666667...","[240.0, 144.0, 240.0, 240.0, 144.0, 240.0, 144...","[2165.0, 2165.0, 1299.0, 1299.0, 2165.0, 2165....","[117.6, 117.66666666666667]","[77.0, 77.2, 77.0, 45.333333333333336, 77.25]",...,"[813.6666666666666, 814.0]",[551.0],[1180.0],[1680.3333333333333],[1582.0],[1696.0],[138.55555555555554],[1626.25],[226.0],[107.42857142857143]
profit,"[-0.47, -0.47, 0.12, -0.47, 0.12, 0.12, 0.12, ...","[0.34, 0.34, 0.34, 0.34]","[0.13, 0.13, 0.48, 0.48, 0.48, 0.48, 0.13, 0.1...","[-0.08, 0.35, -0.08, -0.08]","[0.4, 0.4, 0.4, 0.0, 0.4, 0.0, 0.4]","[0.34, -0.1, 0.34, 0.34]","[0.36, -0.07, 0.36, 0.36, -0.07, 0.36, -0.07, ...","[0.31, 0.31, -0.15, -0.15, 0.31, 0.31, 0.23, 0...","[0.08, 0.08]","[0.22, 0.22, 0.22, -0.47, 0.22]",...,"[0.34, 0.34]",[0.03],[0.33],[0.31],[0.13],[-0.35],[0.11],[-0.15],[0.0],[0.12]


In [30]:
# 函数功能：生成未购买的记录
category_dict = get_category_dict(raw_df)
product_time_dict = get_product_time_feature(raw_df)

fail_dict = {'name':[], 'brand':[], 'days':[], 'deliver_way':[], 'discount':[], 'category':[], 'sub_category':[],
             'price':[],'user_type':[], 'user_city':[], 'user_province':[], 'user_area':[],'profit':[],'sell_num':[],
             'label':[]}
for i in range(len(raw_df['订单 Id'])):
    product_name = raw_df['产品名称'][i]
    
    #获取产品订单时间和同类型的其他产品
    index, product_category = get_name(product_name)
    other_product_list = list(category_dict[product_category])
    other_product_list.remove(product_name)
    if len(other_product_list) == 0:
        continue
    order_time = raw_df['订单日期'][i]
    
    #获取时间最邻近的其他产品的订单
    other_product_near_time = {'name':[], 'days':[], 'index':[]}
    for other_product in other_product_list:
        other_product_near_time['name'].append(other_product)
        other_product_feature = product_time_dict[other_product]
        min_days = 10000
        min_index = 100
        for op_order_time in other_product_feature['order_time']:
            if op_order_time > order_time:
                days = (op_order_time - order_time).days
            else:
                days = (order_time - op_order_time).days
            if days < min_days:
                min_days = days
                min_index = other_product_feature['order_time'].index(op_order_time)
        other_product_near_time['days'].append(min_days)
        other_product_near_time['index'].append(min_index)
    
    #选取所有其他产品中最邻近的作为伪造数据
    all_days = other_product_near_time['days']
    for min_days in all_days:
        if min_days >= 30:
            continue
        md_index = all_days.index(min_days)
        time_index = other_product_near_time['index'][md_index]

        #找到目标特征
        op_name = other_product_near_time['name'][md_index]
        op_order_time = product_time_dict[op_name]['order_time'][time_index]
        op_delivery_way = product_time_dict[op_name]['deliver_way'][time_index]
        op_discount = product_time_dict[op_name]['discount'][time_index]
        op_days = product_time_dict[op_name]['days'][time_index]
        op_price = product_time_dict[op_name]['price'][time_index]
        op_profit = product_time_dict[op_name]['profit'][time_index]

        #插入虚假数据
        #fail_dict['fake_days'].append(min_days)
        fail_dict['name'].append(op_name)
        index, product_category = get_name(op_name)
        fail_dict['brand'].append(op_name[:index])
        #fail_dict['order_time'].append(op_order_time)
        fail_dict['days'].append(op_days)
        fail_dict['deliver_way'].append(op_delivery_way)
        fail_dict['discount'].append(op_discount)
        fail_dict['price'].append(op_price)
        fail_dict['profit'].append(op_profit)
        fail_dict['category'].append(raw_df['类别'][i])
        fail_dict['sub_category'].append(raw_df['子类别'][i])
        fail_dict['user_type'].append(raw_df['细分'][i])
        fail_dict['user_area'].append(raw_df['地区'][i])
        fail_dict['user_province'].append(raw_df['省/自治区'][i])
        fail_dict['user_city'].append(raw_df['城市'][i])
        fail_dict['sell_num'].append(raw_df['数量'][i])
        fail_dict['label'].append(0)

In [14]:
# 函数功能：生成未购买的记录
category_dict = get_category_dict(raw_df)
product_time_dict = get_product_time_feature(raw_df)

fail_dict = {'name':[], 'brand':[], 'days':[], 'deliver_way':[], 'discount':[], 'category':[], 'sub_category':[],
             'price':[],'user_type':[], 'user_city':[], 'user_province':[], 'user_area':[],'label':[]}
for i in range(len(raw_df['订单 Id'])):
    product_name = raw_df['产品名称'][i]
    
    #获取产品订单时间和同类型的其他产品
    index, product_category = get_name(product_name)
    other_product_list = list(category_dict[product_category])
    other_product_list.remove(product_name)
    if len(other_product_list) == 0:
        continue
    order_time = raw_df['订单日期'][i]
    
    #获取时间最邻近的其他产品的订单
    other_product_near_time = {'name':[], 'days':[], 'index':[]}
    for other_product in other_product_list:
        other_product_near_time['name'].append(other_product)
        other_product_feature = product_time_dict[other_product]
        min_days = 10000
        min_index = 100
        for op_order_time in other_product_feature['order_time']:
            if op_order_time > order_time:
                days = (op_order_time - order_time).days
            else:
                days = (order_time - op_order_time).days
            if days < min_days:
                min_days = days
                min_index = other_product_feature['order_time'].index(op_order_time)
        other_product_near_time['days'].append(min_days)
        other_product_near_time['index'].append(min_index)
    
    #选取所有其他产品中最邻近的作为伪造数据
    all_days = other_product_near_time['days']
    min_days = min(all_days)
    if min_days >= 10:
        continue
    md_index = all_days.index(min_days)
    time_index = other_product_near_time['index'][md_index]
    
    #找到目标特征
    op_name = other_product_near_time['name'][md_index]
    op_order_time = product_time_dict[op_name]['order_time'][time_index]
    op_delivery_way = product_time_dict[op_name]['deliver_way'][time_index]
    op_discount = product_time_dict[op_name]['discount'][time_index]
    op_days = product_time_dict[op_name]['days'][time_index]
    op_price = product_time_dict[op_name]['price'][time_index]
    
    #插入虚假数据
    #fail_dict['fake_days'].append(min_days)
    fail_dict['name'].append(op_name)
    index, product_category = get_name(op_name)
    fail_dict['brand'].append(op_name[:index])
    #fail_dict['order_time'].append(op_order_time)
    fail_dict['days'].append(op_days)
    fail_dict['deliver_way'].append(op_delivery_way)
    fail_dict['discount'].append(op_discount)
    fail_dict['price'].append(op_price)
    fail_dict['category'].append(raw_df['类别'][i])
    fail_dict['sub_category'].append(raw_df['子类别'][i])
    fail_dict['user_type'].append(raw_df['细分'][i])
    fail_dict['user_area'].append(raw_df['地区'][i])
    fail_dict['user_province'].append(raw_df['省/自治区'][i])
    fail_dict['user_city'].append(raw_df['城市'][i])
    fail_dict['label'].append(0)

In [31]:
pd.DataFrame(data=fail_dict)

,name,brand,days,deliver_way,discount,category,sub_category,price,user_type,user_city,user_province,user_area,profit,sell_num,label
0,"Acco 孔加固材料, 回收",Acco,5,标准级,0.4,办公用品,装订机,19.666667,消费者,内江,四川,西南,-0.64,2,0
1,"Ibico 孔加固材料, 回收",Ibico,5,标准级,0.0,办公用品,装订机,34.250000,消费者,内江,四川,西南,0.34,2,0
2,"Wilson Jones 订书机, 实惠",Wilson Jones,4,标准级,0.4,办公用品,装订机,137.000000,消费者,景德镇,江西,华东,0.05,2,0
3,"Harbour Creations 扶手椅, 可调",Harbour Creations,6,标准级,0.4,家具,椅子,1333.000000,消费者,景德镇,江西,华东,0.08,4,0
4,"Eaton 计划信息表, 多色",Eaton,4,二级,0.0,办公用品,纸张,102.500000,消费者,景德镇,江西,华东,0.25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8257,"Fellowes 盘, 蓝色",Fellowes,4,标准级,0.0,办公用品,收纳具,267.846154,消费者,义乌,浙江,华东,0.23,2,0
8258,"Office Star 折叠椅, 红色",Office Star,1,一级,0.0,家具,椅子,428.000000,消费者,义乌,浙江,华东,0.19,1,0
8259,"Enermax 路由器, 耐用",Enermax,2,一级,0.4,技术,配件,723.916667,消费者,义乌,浙江,华东,-0.63,7,0
8260,"Memorex 路由器, 耐用",Memorex,4,标准级,0.0,技术,配件,1150.000000,消费者,义乌,浙江,华东,0.41,7,0


In [32]:
# 形成真实数据集
success_dict = {'name':[], 'brand':[], 'days':[], 'deliver_way':[], 'discount':[], 'category':[], 'sub_category':[],
             'price':[],'user_type':[], 'user_city':[], 'user_province':[], 'user_area':[],'profit':[],'sell_num':[],'label':[]}
for i in range(len(raw_df['订单 Id'])):
    success_dict['name'].append(raw_df['产品名称'][i])
    success_dict['brand'].append(raw_df['制造商'][i])
    success_dict['days'].append((raw_df['发货日期'][i] - raw_df['订单日期'][i]).days)
    success_dict['deliver_way'].append(raw_df['邮寄方式'][i])
    success_dict['discount'].append(raw_df['折扣'][i])
    success_dict['category'].append(raw_df['类别'][i])
    success_dict['sub_category'].append(raw_df['子类别'][i])
    success_dict['price'].append(raw_df['销售额'][i] / raw_df['数量'][i])
    success_dict['user_type'].append(raw_df['细分'][i])
    success_dict['user_area'].append(raw_df['地区'][i])
    success_dict['user_city'].append(raw_df['城市'][i])
    success_dict['user_province'].append(raw_df['省/自治区'][i])
    success_dict['profit'].append(raw_df['利润率'][i])
    success_dict['sell_num'].append(raw_df['数量'][i])
    success_dict['label'].append(1)

In [33]:
all_data_dict = {'name':[], 'brand':[], 'days':[], 'deliver_way':[], 'discount':[], 'category':[], 'sub_category':[],
             'price':[],'user_type':[], 'user_city':[], 'user_province':[], 'user_area':[],'profit':[],'sell_num':[],'label':[]}
for i in range(len(fail_dict['name'])):
    for key in all_data_dict.keys():
        all_data_dict[key].append(fail_dict[key][i])
for i in range(len(success_dict['name'])):
    for key in all_data_dict.keys():
        all_data_dict[key].append(success_dict[key][i])

In [34]:
df = pd.DataFrame(data=all_data_dict)
df.to_csv('./Data/deal_add_all_below_30.csv',index=None)

In [35]:
pd.DataFrame(data=all_data_dict)

,name,brand,days,deliver_way,discount,category,sub_category,price,user_type,user_city,user_province,user_area,profit,sell_num,label
0,"Acco 孔加固材料, 回收",Acco,5,标准级,0.4,办公用品,装订机,19.666667,消费者,内江,四川,西南,-0.64,2,0
1,"Ibico 孔加固材料, 回收",Ibico,5,标准级,0.0,办公用品,装订机,34.250000,消费者,内江,四川,西南,0.34,2,0
2,"Wilson Jones 订书机, 实惠",Wilson Jones,4,标准级,0.4,办公用品,装订机,137.000000,消费者,景德镇,江西,华东,0.05,2,0
3,"Harbour Creations 扶手椅, 可调",Harbour Creations,6,标准级,0.4,家具,椅子,1333.000000,消费者,景德镇,江西,华东,0.08,4,0
4,"Eaton 计划信息表, 多色",Eaton,4,二级,0.0,办公用品,纸张,102.500000,消费者,景德镇,江西,华东,0.25,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18192,"SanDisk 记忆卡, 实惠",SanDisk,5,标准级,0.4,技术,配件,314.666667,消费者,义乌,浙江,华东,-0.33,3,1
18193,"Eldon 盘, 蓝色",Eldon,5,标准级,0.0,办公用品,收纳具,224.000000,消费者,义乌,浙江,华东,0.08,2,1
18194,"SAFCO 折叠椅, 红色",Safco,5,标准级,0.4,家具,椅子,240.000000,消费者,义乌,浙江,华东,-0.37,1,1
18195,"罗技 路由器, 耐用",罗技,5,标准级,0.4,技术,配件,693.142857,消费者,义乌,浙江,华东,-0.33,7,1
